In [2]:
using JuMP
import Pkg
Pkg.add("Gurobi")
import Gurobi
using CSV, DataFrames
using Random
import Test


   Resolving package versions...
  No Changes to `C:\Users\Joe\OneDrive\Documents\GitHub\LPGenie\Project.toml`
  No Changes to `C:\Users\Joe\OneDrive\Documents\GitHub\LPGenie\Manifest.toml`


In [5]:
data = CSV.read("RoofDummyData(4product12month).csv",DataFrame)
print(data)
#nrow(data) = 24
#ncol(data) = 5
data1= CSV.read("dummydata1.csv",DataFrame)
data1
#nrow(data1)=4
#ncol(data1)=4




12×5 DataFrame
 Row │ Column1    XC60_Gent  XC60_China  Kuga   Mondeo 
     │ String15   Int64      Int64       Int64  Int64  
─────┼─────────────────────────────────────────────────
   1 │ 1/1/2024        5668        5952   2012     924
   2 │ 1/2/2024        3916        3744   2128     800
   3 │ 1/3/2024        5312        1552   2632     964
   4 │ 1/4/2024        6720        3032   1740     768
   5 │ 1/5/2024        4092        2372   1540     648
   6 │ 1/6/2024        3108        2292   2292     848
   7 │ 1/7/2024        4656        1568   2920     828
   8 │ 1/8/2024        4772         612   3256     764
   9 │ 1/9/2024        3408        3988   2288     720
  10 │ 1/10/2024       2936        2540   2424     448
  11 │ 1/11/2024       8284        6680   2228     700
  12 │ 1/12/2024       4516        3260   2660     568

Row,Column1,worker 1,worker 2,worker 3
,String15,Int64,Int64,Int64
1,XC60_Gent,44,168,63
2,XC60_China,138,154,92
3,Kuga,132,110,161
4,Mondeo,115,0,66


In [3]:
Random.seed!(1234) # set seed

num_products_p = ncol(data) - 1  #to not include the time columnindex
time_horizon_T = nrow(data)
product_names = names(data)[2:end]
num_workertype_W=ncol(data1)-1
worker_names=names(data1)[2:end]
println(num_workertype_W)


#Initialise demand
#demand_D= [] #Initialise 
#for col in 2:ncol(data) #Extract each column as a seperate array and store it
#    push!(demand_D, data[:,col])
#end
#println(demand_D)

demand_data = Matrix(data[:, 2:end])
demand_D = transpose(demand_data)
println(demand_D)

#Initialise workdays 
workdays_n = [] #Initialise 
for row in 1:time_horizon_T
    push!(workdays_n, rand(20:23)) #push a random workday value between 20 to 23
end
println(workdays_n)

#Initialise productivity
#productionrate = []
#for col in 1:ncol(data1)
#    push!(productionrate, collect(data1[col, 2:end]))
#end
#println(productionrate_K)

df = select(data1, Not(:Column1))
productionrate_K = Matrix(df)
println(productionrate_K)


# Initialise Costs
cost_hiring_cH = 5882.60    # Hiring cost per worker
cost_firing_cF = 857.23    # Firing cost per worker
cost_inventory_cI = 9.05 # Inventory holding cost per unit
cost_labour_cR = 233.30  # Cost of Labour per production unit
cost_overtime_cO = 349.89 # Cost of Overtime per overtime unit
cost_backlogging_cB = 135.31  #Cost of Backlogging per overtime unit 
cost_idle_cU= 20 #Cost of Idle per worker
cost_subcontract_cS=250 #cost of Subcontract 

#code ends here

3
[5668 3916 5312 6720 4092 3108 4656 4772 3408 2936 8284 4516; 5952 3744 1552 3032 2372 2292 1568 612 3988 2540 6680 3260; 2012 2128 2632 1740 1540 2292 2920 3256 2288 2424 2228 2660; 924 800 964 768 648 848 828 764 720 448 700 568]
Any[21, 22, 20, 23, 21, 21, 23, 23, 21, 22, 22, 22]
[44 168 63; 138 154 92; 132 110 161; 115 0 66]


250

In [4]:
# Initialize the model
model = Model(Gurobi.Optimizer)

const ENV["GRB_LICENSE_FILE"] = joinpath("gurobi.lic")

# Variables
@variable(model, workerlevel_L[1:num_workertype_W,1:time_horizon_T] >= 0)       # Workers
@variable(model, hired_H[1:num_workertype_W,1:time_horizon_T] >= 0)       # Hired workers
@variable(model, fired_F[1:num_workertype_W,1:time_horizon_T] >= 0)       # Fired workers
@variable(model, inventory_I[1:num_products_p,1:time_horizon_T] >= 0)       # Inventory as integer variables
@variable(model, production_P[1:num_products_p,1:time_horizon_T] >= 0)       # Production
@variable(model, overtime_O[1:num_products_p,1:time_horizon_T] >= 0)       # Overtime
@variable(model, backlogging_B[1:num_products_p,1:time_horizon_T] >= 0)        # Production
@variable(model, idle_U[1:num_products_p,1:time_horizon_T] >= 0)         #Idle
@variable(model, subcontract_S[1:num_products_p,1:time_horizon_T] >= 0)         #Subcontract

@objective(model, Min,
    @objective(model, Min, 
    sum(cost_hiring_cH *hired_H + cost_firing_cF*fired_F for w in num_workertype_W, t in time_horizon_T) +
    sum(cost_inventory_cI*inventory_I + cost_labour_cR*production_P 
    + cost_overtime_cO*overtime_O + cost_idle_cU*idle_U+ cost_subcontract_cS*subcontract_S + cost_backlogging_cB*backlogging_B for p in num_products_p, t in time_horizon_T)
))

# Constraints

for w in 1:num_workertype_W
    @constraint(model, workerlevel_L[w, 1] == hired_H[w, 1] - fired_F[w, 1])
end

#P equations
for w in 1:num_workertype_W
    for t in 2:time_horizon_T
        @constraint(model, workerlevel_L[w,t] == workerlevel_L[w,t-1] + hired_H[w,t] - fired_F[w,t])
    end
end

# for p in in 1:num_products_p
#     for t in 1:time_horizon_T
#         @constraint(model, production_P[p,t] ==  sum(productionrate_K[p,w] * workerlevel_L[w, t] for w in num_workertype_W) + overtime_O[p,t] - idle_U[p,t])
#     end
# end

for p in 1:num_products_p
    @constraint(model, inventory_I[p, 1] - backlogging_B[p, 1] == production_P[p, 1] + subcontract_S[p, 1] - demand_D[p,w])
end

for p in 1:num_products_p
    for t in 2:time_horizon_T
        @constraint(model, inventory_I[p, t] - backlogging_B[p, t] == inventory_I[p, t-1] + backlogging_B[p, t-1] + production_P[p, t] + subcontract_S[p, t] - demand_D[p,t])
    end
end
# Solve the model
optimize!(model)

#retrieve Values
println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_W)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528347
Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg


LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(12)), b has dims (Base.OneTo(4), Base.OneTo(12)), mismatch at 1

In [ ]:
println(size(productionrate_K))   # Expected: (num_products_p, num_workertype_W)
println(size(demand_D))           # Expected: (num_products_p, time_horizon_T)
println(size(workerlevel_L))      # Expected: (num_workertype_W, time_horizon_T)
println(size(production_P))
println(demand_D)
println(typeof(demand_D))  # Should be Array{Int64, 2}
println(length(demand_D))
print(typeof(productionrate_K))
println(productionrate_K)
println(productionrate_K[2,2])

In [62]:
time_horizion_T=12
num_workertype_W=3
println(production_P)
println(productionrate_K)
size(productionrate_K)
println(demand_D)
println(demand_D[3,2])
size(demand_D)

VariableRef[production_P[1,1] production_P[1,2] production_P[1,3] production_P[1,4] production_P[1,5] production_P[1,6] production_P[1,7] production_P[1,8] production_P[1,9] production_P[1,10] production_P[1,11] production_P[1,12]; production_P[2,1] production_P[2,2] production_P[2,3] production_P[2,4] production_P[2,5] production_P[2,6] production_P[2,7] production_P[2,8] production_P[2,9] production_P[2,10] production_P[2,11] production_P[2,12]; production_P[3,1] production_P[3,2] production_P[3,3] production_P[3,4] production_P[3,5] production_P[3,6] production_P[3,7] production_P[3,8] production_P[3,9] production_P[3,10] production_P[3,11] production_P[3,12]; production_P[4,1] production_P[4,2] production_P[4,3] production_P[4,4] production_P[4,5] production_P[4,6] production_P[4,7] production_P[4,8] production_P[4,9] production_P[4,10] production_P[4,11] production_P[4,12]]
[44 168 63; 138 154 92; 132 110 161; 115 0 66]
[5668 3916 5312 6720 4092 3108 4656 4772 3408 2936 8284 4516;

(4, 12)

In [59]:
time_horizion_T=12
num_workertype_W=3
num_products_p=4

# Initialise Costs
cost_hiring_cH = 5882.30   # Hiring cost per worker
cost_firing_cF = 857.23    # Firing cost per worker
cost_inventory_cI = 9.05 # Inventory holding cost per unit
cost_labour_cR = 200.30  # Cost of Labour per production unit
cost_overtime_cO = 349.89 # Cost of Overtime per overtime unit
cost_backlogging_cB = 1000  #Cost of Backlogging per overtime unit 
cost_idle_cU= 1000 #Cost of Idle per worker
cost_subcontract_cS= 1000 #cost of Subcontract 

1000

In [60]:
#Our Attempt

using JuMP
using Gurobi

# Define the model
model = Model(Gurobi.Optimizer)


# Variables
@variable(model, workerlevel_L[1:num_workertype_W, 1:time_horizon_T] >= 0, Int) # Workers
@variable(model, hired_H[1:num_workertype_W, 1:time_horizon_T] >= 0, Int)       # Hired workers
@variable(model, fired_F[1:num_workertype_W, 1:time_horizon_T] >= 0, Int)       # Fired workers
@variable(model, inventory_I[1:num_products_p, 1:time_horizon_T] >= 0, Int)     # Inventory
@variable(model, production_P[1:num_products_p, 1:time_horizon_T] >= 0, Int)    # Production
@variable(model, overtime_O[1:num_products_p, 1:time_horizon_T] >= 0, Int)      # Overtime
@variable(model, backlogging_B[1:num_products_p, 1:time_horizon_T] >= 0, Int)   # Backlogging
@variable(model, idle_U[1:num_products_p, 1:time_horizon_T] >= 0, Int)          # Idle
@variable(model, subcontract_S[1:num_products_p, 1:time_horizon_T] >= 0, Int)   # Subcontract

# Objective
@objective(model, Min,
    sum(cost_hiring_cH * hired_H[w, t] + cost_firing_cF * fired_F[w, t] for w in 1:num_workertype_W, t in 1:time_horizon_T) +
    sum(cost_inventory_cI * inventory_I[p, t] + cost_labour_cR * production_P[p, t] +
        cost_overtime_cO * overtime_O[p, t] + cost_idle_cU * idle_U[p, t] +
        cost_subcontract_cS * subcontract_S[p, t] + cost_backlogging_cB * backlogging_B[p, t]
        for p in 1:num_products_p, t in 1:time_horizon_T)
)

# Constraints

# Worker level constraints for period 1
for w in 1:num_workertype_W
    @constraint(model, workerlevel_L[w, 1] == hired_H[w, 1] - fired_F[w, 1])
end

# Worker level constraints for subsequent periods
for w in 1:num_workertype_W
    for t in 2:time_horizon_T
        @constraint(model, workerlevel_L[w, t] == workerlevel_L[w, t-1] + hired_H[w, t] - fired_F[w, t])
        @constraint(model, 0.1*workerlevel_L[w, t] >= hired_H[w, t] + fired_F[w, t])
    end
end

# Production constraints
for p in 1:num_products_p
    for t in 1:time_horizon_T
        @constraint(model, production_P[p, t] == sum(productionrate_K[p, w] * workerlevel_L[w, t] for w in 1:num_workertype_W) + overtime_O[p, t] - idle_U[p, t])
    end
end

# Inventory and backlogging constraints for period 1
for p in 1:num_products_p
    @constraint(model, inventory_I[p, 1] - backlogging_B[p, 1] == production_P[p, 1] + subcontract_S[p, 1] - demand_D[p, 1])
end

# Inventory and backlogging constraints for subsequent periods
for p in 1:num_products_p
    for t in 2:time_horizon_T
        @constraint(model, inventory_I[p, t] - backlogging_B[p, t] == inventory_I[p, t-1] + backlogging_B[p, t-1] + production_P[p, t] + subcontract_S[p, t] - demand_D[p, t])
    end
end

# Solve the model
optimize!(model)

println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_L)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))
println("Idle: ", round.(Int,value.(idle_U)))
println("Subcontract: ", round.(Int,value.(subcontract_S)))
println(termination_status(model))

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528347
Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Optimize a model with 165 rows, 396 columns and 796 nonzeros
Model fingerprint: 0x5514e710
Variable types: 0 continuous, 396 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [9e+00, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+03]
Found heuristic solution: objective 1.320800e+08
Presolve removed 3 rows and 114 columns
Presolve time: 0.00s
Presolved: 162 rows, 282 columns, 679 nonzero

In [1]:
#Group 2 Attempt

using JuMP
using Gurobi

# Define the model
model = Model(Gurobi.Optimizer)

# Variables
@variable(model, workerlevel_L[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int) # Workers
@variable(model, hired_H[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)       # Hired workers
@variable(model, fired_F[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)       # Fired workers
@variable(model, inventory_I[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)     # Inventory
@variable(model, production_P[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)    # Production
@variable(model, overtime_O[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)      # Overtime
@variable(model, backlogging_B[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)   # Backlogging
@variable(model, idle_U[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)          # Idle
@variable(model, subcontract_S[1:num_workertype_W,1:num_products_p, 1:time_horizon_T] >= 0, Int)   # Subcontract

# Objective
@objective(model, Min,
    sum(cost_hiring_cH * hired_H + cost_firing_cF * fired_F +
        cost_inventory_cI * inventory_I + cost_labour_cR * production_P +
        cost_overtime_cO * overtime_O + cost_idle_cU * idle_U +
        cost_subcontract_cS * subcontract_S + cost_backlogging_cB * backlogging_B))

# Constraints

# Worker level constraints for period 1
for w in 1:num_workertype_W
    for p in num_products_p
        @constraint(model, workerlevel_L[w,p,1] == hired_H[w,p,1] - fired_F[w,p,1])
    end    
end

# Worker level constraints for subsequent periods
for w in 1:num_workertype_W
    for p in num_products_p
        for t in 2:time_horizon_T
            @constraint(model, workerlevel_L[w,p,t] == workerlevel_L[w,p,t-1] + hired_H[w,p,t] - fired_F[w,p,t])
            # @constraint(model, 0.1*workerlevel_L[w,p,t] >= hired_H[w,p,t] + fired_F[w,p,t]) #additional constraint
        end    
    end
end

# Production constraints
for w in 1:num_workertype_W
    for p in num_products_p
        for t in 2:time_horizon_T
            @constraint(model, production_P[w,p,t] == workerlevel_L[w,p,t] + overtime_O[w,p,t] - idle_U[w,p,t])
        end    
    end
end

# Inventory and backlogging constraints for period 1
for w in 1:num_workertype_W
    for p in 1:num_products_p
        @constraint(model, inventory_I[w,p,1] - backlogging_B[w,p,1] == production_P[w,p,1] + subcontract_S[w,p,1] - demand_D[w,p,1])
    end    
end

# Inventory and backlogging constraints for subsequent periods
for w in 1:num_workertype_W
    for p in 1:num_products_p
        for t in 2:time_horizon_T
            @constraint(model, inventory_I[w,p,t] - backlogging_B[w,p,t] == inventory_I[w,p,t-1] + backlogging_B[w,p,t-1] + production_P[w,p,t] + subcontract_S[w,p,t] - demand_D[p,t])
        end    
    end
end

# Solve the model
optimize!(model)

println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_L)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))
println("Idle: ", round.(Int,value.(idle_U)))
println("Subcontract: ", round.(Int,value.(subcontract_S)))
println(value.(workerlevel_L))

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528347
Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg


LoadError: At In[1]:10: `@variable(model, workerlevel_L[1:num_workertype_W, 1:num_products_p, 1:time_horizon_T] >= 0, Int)`: unexpected error parsing reference set: 1:num_workertype_W

In [22]:
productivity_K = [20,21,22,23]

4-element Vector{Int64}:
 20
 21
 22
 23

In [52]:
#Initial Attempt

# Initialize the model
model = Model(Gurobi.Optimizer)

# Variables
@variable(model, workerlevel_W[1:num_products_p,1:time_horizon_T] >= 0)       # Workers
@variable(model, hired_H[1:num_products_p,1:time_horizon_T] >= 0)       # Hired workers
@variable(model, fired_F[1:num_products_p,1:time_horizon_T] >= 0)       # Fired workers
@variable(model, inventory_I[1:num_products_p,1:time_horizon_T] >= 0)       # Inventory as integer variables
@variable(model, production_P[1:num_products_p,1:time_horizon_T] >= 0)       # Production
@variable(model, overtime_O[1:num_products_p,1:time_horizon_T] >= 0)       # Overtime
@variable(model, backlogging_B[1:num_products_p,1:time_horizon_T] >= 0)        # Production

# Objective function: Minimize total cost
@objective(model, Min, sum(cost_hiring_cH*hired_H + cost_firing_cF*fired_F 
+ cost_inventory_cI*inventory_I + cost_labour_cR*production_P 
+ cost_overtime_cO*overtime_O + cost_backlogging_cB*backlogging_B))

# Constraints

#only for period 1, W annd I-B eqns
for p in 1:num_products_p
    @constraint(model, workerlevel_W[p,1] == hired_H[p,1] - fired_F[p,1])
    @constraint(model, inventory_I[p,1] - backlogging_B[p,1] == production_P[p,1] - demand_D[p,1])
end    

#for rest of period, W and I-B equations
for p in 1:num_products_p
    for t in 2:time_horizon_T
        @constraint(model, workerlevel_W[p,t] 
        == workerlevel_W[p,t-1] + hired_H[p,t] - fired_F[p,t])
        @constraint(model, inventory_I[p,t] - backlogging_B[p,t] 
        == inventory_I[p,t-1] - backlogging_B[p,t-1] 
        + production_P[p,t] - demand_D[p,t])
    end
end

#P equations & (tao values, to be inserted)
for p in 1:num_products_p
    for t in 1:time_horizon_T
        @constraint(model, production_P[p,t] 
        == productivity_K[p] * workdays_n[t] * workerlevel_W[p,t])
    end    
end

# Solve the model
optimize!(model)

println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_W)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))
println("Idle: ", round.(Int,value.(idle_U)))
println("Subcontract: ", round.(Int,value.(subcontract_S)))
println(termination_status(model))

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528347
Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Optimize a model with 144 rows, 336 columns and 516 nonzeros
Model fingerprint: 0x71be91bc
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [9e+00, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+03]
Presolve removed 56 rows and 156 columns
Presolve time: 0.00s
Presolved: 88 rows, 180 columns, 308 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1112113e+06   1.76210